In [60]:
import pandas as pd
from newspaper import Article
import nltk
import ast
import requests
from tqdm.auto import tqdm
tqdm.pandas()
from bs4 import BeautifulSoup, Comment
import numpy as np

In [61]:
headers = {
    'Access-Control-Allow-Origin': '*',
    'Access-Control-Allow-Methods': 'GET',
    'Access-Control-Allow-Headers': 'Content-Type',
    'Access-Control-Max-Age': '3600',
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0'
    }

In [73]:
df = pd.read_csv('thehimalayan.csv')

In [74]:
# new = pd.read_csv("thehimalayan_articles.csv")

In [75]:
# joined = df[['id', 'created_at', 'urls']].set_index('id').join(new[['id', 'article']].set_index('id')).reset_index()

In [76]:
# joined

In [77]:
# joined = joined[joined.article!="404"]

In [78]:
# joined

In [79]:
# joined.to_csv('thehimalayan.csv', index=False)

In [80]:
df.shape

(32485, 4)

In [81]:
df.head()

,id,created_at,urls,article
0,1386731977542455296,2021-04-26 23:05:42+05:45,https://bit.ly/3gBXQpn,"NEW DELHI, APRIL 26\n\nIndia ordered its armed..."
1,1386727294434254857,2021-04-26 22:47:05+05:45,https://bit.ly/2RWCAjG,"KATHMANDU, APRIL 26\n\nAs Nepal is currently g..."
2,1386722788099170307,2021-04-26 22:29:11+05:45,https://bit.ly/2PpCtwj,"DHANGADHI, APRIL 26\n\nFollowing the rise in t..."
3,1386713496511975428,2021-04-26 21:52:16+05:45,https://bit.ly/3evKsAu,"MELBOURNE, APRIL 26\n\nAustralia leg-spinner A..."
4,1386709296264658950,2021-04-26 21:35:34+05:45,https://bit.ly/2R0XC0g,"KATHMANDU, APRIL 26\n\n​​​​​​​Authorities in N..."


In [84]:
df.tail()

,id,created_at,urls,article
32480,947678773498933249,2018-01-01 09:45:01+05:45,https://wp.me/p6iC0n-1YTA,NaN
32481,947677332487020544,2018-01-01 09:39:17+05:45,https://wp.me/p6iC0n-1YUL,NaN
32482,947674994766516224,2018-01-01 09:30:00+05:45,https://wp.me/p6iC0n-1YU4,NaN
32483,947668683454799878,2018-01-01 09:04:55+05:45,https://wp.me/p6iC0n-1YU3,NaN
32484,947656457888993281,2018-01-01 08:16:20+05:45,https://wp.me/p6iC0n-1YTU,NaN


## Chunk

In [87]:
def article_ktmpost(row):
    try:
        if (row.article != row.article):
            url = row.urls
            req = requests.get(url, headers)
            soup = BeautifulSoup(req.content, 'html.parser')
            result = soup.find("div", {"class": "col-sm-8"})
            return result.text
        return row.article
    except Exception as e:
        print (str(e))
        return 404
    
# for himalayan times
from newspaper import Config

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:78.0) Gecko/20100101 Firefox/78.0'

config = Config()
config.browser_user_agent = user_agent
config.request_timeout = 3
    
def article(row):
    try:
        if (row.article != row.article):
            url = row.urls
            article = Article(url, config=config)
            article.download()
            article.parse()
            return article.text
        return row.article
    except Exception as e:
        print (str(e))
        return 404  

In [ ]:
file_path = "thehimalayan.csv"
result_file_path = "thehimalayan_continue.csv"

pd.read_csv(file_path, nrows=1).head(0).to_csv(result_file_path)
dataframe = pd.read_csv(file_path, iterator=True, chunksize=1000)

for chunk in dataframe:
   chunk['article'] = chunk.progress_apply(article, axis=1)
   chunk.to_csv(result_file_path, mode='a', header=None)

Article `download()` failed with HTTPConnectionPool(host='tht.wpengine.com', port=80): Max retries exceeded with url: /?p=641835 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7ff0bc63b520>, 'Connection to tht.wpengine.com timed out. (connect timeout=3)')) on URL https://wp.me/p6iC0n-2GYb
Article `download()` failed with HTTPConnectionPool(host='tht.wpengine.com', port=80): Max retries exceeded with url: /?p=641762 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7ff0bc6a2160>, 'Connection to tht.wpengine.com timed out. (connect timeout=3)')) on URL https://wp.me/p6iC0n-2GX0
Article `download()` failed with HTTPConnectionPool(host='tht.wpengine.com', port=80): Max retries exceeded with url: /?p=641838 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7ff0bc63bc70>, 'Connection to tht.wpengine.com timed out. (connect timeout=3)')) on URL https://wp.me/p6iC0n-2GYe
Article `download()` failed with HTTP

Article `download()` failed with HTTPConnectionPool(host='tht.wpengine.com', port=80): Max retries exceeded with url: /?p=641424 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7ff0bc63b160>, 'Connection to tht.wpengine.com timed out. (connect timeout=3)')) on URL https://wp.me/p6iC0n-2GRy
Article `download()` failed with HTTPConnectionPool(host='tht.wpengine.com', port=80): Max retries exceeded with url: /?p=641412 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7ff0bc6368b0>, 'Connection to tht.wpengine.com timed out. (connect timeout=3)')) on URL https://wp.me/p6iC0n-2GRm
Article `download()` failed with HTTPConnectionPool(host='tht.wpengine.com', port=80): Max retries exceeded with url: /?p=641400 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x7ff0bc69ca90>, 'Connection to tht.wpengine.com timed out. (connect timeout=3)')) on URL https://wp.me/p6iC0n-2GRa


In [55]:
new = pd.read_csv(result_file_path)

In [56]:
new.shape

(41172, 5)

In [59]:
new[new.article!='404'][['id', 'created_at', 'urls', 'article']].to_csv('RepublicaNepal.csv', index=False)